--- Day 3: Mull It Over ---

"Our computers are having issues, so I have no idea if we have any Chief Historians in stock! You're welcome to check the warehouse, though," says the mildly flustered shopkeeper at the North Pole Toboggan Rental Shop. The Historians head out to take a look.

The shopkeeper turns to you. "Any chance you can see why our computers are having issues again?"

The computer appears to be trying to run a program, but its memory (your puzzle input) is corrupted. All of the instructions have been jumbled up!

It seems like the goal of the program is just to multiply some numbers. It does that with instructions like mul(X,Y), where X and Y are each 1-3 digit numbers. For instance, mul(44,46) multiplies 44 by 46 to get a result of 2024. Similarly, mul(123,4) would multiply 123 by 4.

However, because the program's memory has been corrupted, there are also many invalid characters that should be ignored, even if they look like part of a mul instruction. Sequences like mul(4*, mul(6,9!, ?(12,34), or mul ( 2 , 4 ) do nothing.

For example, consider the following section of corrupted memory:

xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))

Only the four highlighted sections are real mul instructions. Adding up the result of each instruction produces 161 (2*4 + 5*5 + 11*8 + 8*5).

Scan the corrupted memory for uncorrupted mul instructions. What do you get if you add up all of the results of the multiplications?


In [5]:
import re

class MultiplicationParser:
    def __init__(self):
        # Pattern matches 'mul' followed by 1-3 digits, comma, and 1-3 digits in parentheses
        self.pattern = r'mul\((\d{1,3}),(\d{1,3})\)'
        
    def process_file(self, file_path):
        try:
            with open(file_path, 'r') as file:
                content = file.read()
                print(content)
                return self.print_results(content)
        except FileNotFoundError:
            print(f"Błąd: Plik {file_path} nie został znaleziony.")
            return None
        except Exception as e:
            print(f"Wystąpił błąd podczas przetwarzania pliku: {str(e)}")
            return None
        
    def calculate_sum(self, text):
        # Find all matches in the text
        matches = re.finditer(self.pattern, text)
        
        total_sum = 0
        multiplications = []
        
        # Process each match
        for match in matches:
            num1 = int(match.group(1))
            num2 = int(match.group(2))
            result = num1 * num2
            multiplications.append((num1, num2, result))
            total_sum += result
            
        return {
            'multiplications': multiplications,
            'total_sum': total_sum
        }
    
    def print_results(self, text):
        results = self.calculate_sum(text)
        
        print("Znalezione mnożenia:")
        for num1, num2, result in results['multiplications']:
            print(f"{num1} * {num2} = {result}")
        
        print(f"\nSuma wszystkich iloczynów: {results['total_sum']}")
        return results['total_sum']
    
    # Przykład użycia
parser = MultiplicationParser()
text = "xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))"
result = parser.print_results(text)

Znalezione mnożenia:
2 * 4 = 8
5 * 5 = 25
11 * 8 = 88
8 * 5 = 40

Suma wszystkich iloczynów: 161


In [6]:
result = parser.process_file("input.txt")

^+'*>,,why()mul(229,919)&$-#^~mul(187,600)@<select()mul(430,339)<)*/-when()%mul(248,922)~+when()<do()^}%where()@select() what()why()who(809,724)mul(617,192)$*from()what(168,899)mul(333,411)()$select(){+how()%mul(284,904)when();who()mul(218,212)>[#' *+&mul(388,743):~^&;do()when()&^&^mul(415,678)>what(180,378)when()/)!#how()~&do()(((]how()[~{;what()mul(792,328)[;(,why()#mul(767,729)(what()@-why()}who()how()where(373,159),mul(91,503)select()~;where()@+;;++don't()mul(766,411)~'&%what(217,603)>why()mul(528,603);how() &who()mul(418,950)-select()mul(440,425)mul(42,798):what()[^%mul(28,566)from()<%>]//(<mul(167,358)'%](#mul(77,714)mul(748,367)]*mul(124,693);where(156,464)^(^[what()why();do()<>*mul(121,164)+/)@}/why()([mul(94,107)where()why()$what()} ~when()who()when()mul(51,763)&:$)mul(117,974)*%?]%;who()>mul(224,518)@/<*,}mul(681,258)(%{#select()&*who()where()mul(301,302) don't()who()][!from()>mul(410,170)do())what()##mul(873,748)$~}mul(797,721):when()how()[:-where()>mul(959,163)mul(992,12)%/

Your puzzle answer was 173785482.

The first half of this puzzle is complete! It provides one gold star: *
--- Part Two ---

As you scan through the corrupted memory, you notice that some of the conditional statements are also still intact. If you handle some of the uncorrupted conditional statements in the program, you might be able to get an even more accurate result.

There are two new instructions you'll need to handle:

    The do() instruction enables future mul instructions.
    The don't() instruction disables future mul instructions.

Only the most recent do() or don't() instruction applies. At the beginning of the program, mul instructions are enabled.

For example:

xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))

This corrupted memory is similar to the example from before, but this time the mul(5,5) and mul(11,8) instructions are disabled because there is a don't() instruction before them. The other mul instructions function normally, including the one at the end that gets re-enabled by a do() instruction.

This time, the sum of the results is 48 (2*4 + 8*5).

Handle the new instructions; what do you get if you add up all of the results of just the enabled multiplications?


In [7]:
import re

class MultiplicationParser:
    def __init__(self):
        # Pattern matches 'mul' followed by 1-3 digits, comma, and 1-3 digits in parentheses
        self.pattern = r'mul\((\d{1,3}),(\d{1,3})\)'
        self.do_pattern = r'do\(\)'
        self.dont_pattern = r'don\'t\(\)'
        
    def calculate_sum_with_conditions(self, text):
        # Find all matches for mul, do() and don't()
        mul_matches = [(m.start(), 'mul', m) for m in re.finditer(self.pattern, text)]
        do_matches = [(m.start(), 'do', m) for m in re.finditer(self.do_pattern, text)]
        dont_matches = [(m.start(), 'dont', m) for m in re.finditer(self.dont_pattern, text)]
        
        # Add virtual do() at position -1 (beginning of text)
        virtual_do = [(-1, 'do', None)]
        
        # Combine all matches and sort by position
        all_matches = sorted(virtual_do + mul_matches + do_matches + dont_matches, key=lambda x: x[0])
        
        print("Kolejność instrukcji:")
        for pos, type, _ in all_matches:
            print(f"Pozycja {pos}: {type}")
        
        total_sum = 0
        multiplications = []
        enabled = True  # At the beginning, mul instructions are enabled
        
        for pos, type, match in all_matches:
            if type == 'do':
                enabled = True
                print(f"Włączono mnożenia na pozycji {pos}")
            elif type == 'dont':
                enabled = False
                print(f"Wyłączono mnożenia na pozycji {pos}")
            elif type == 'mul' and enabled:
                num1 = int(match.group(1))
                num2 = int(match.group(2))
                result = num1 * num2
                multiplications.append((num1, num2, result, enabled))
                total_sum += result
                print(f"Mnożenie na pozycji {pos}: {num1} * {num2} = {result} (włączone)")
                
        return {
            'multiplications': multiplications,
            'total_sum': total_sum
        }
    
    def print_results_with_conditions(self, text):
        results = self.calculate_sum_with_conditions(text)
        
        print("\nZnalezione mnożenia (tylko włączone):")
        for num1, num2, result, enabled in results['multiplications']:
            print(f"{num1} * {num2} = {result} ({'włączone' if enabled else 'wyłączone'})")
        
        print(f"\nSuma wszystkich włączonych iloczynów: {results['total_sum']}")
        return results['total_sum']

    def process_file(self, file_path):
        try:
            with open(file_path, 'r') as file:
                content = file.read()
                return self.print_results_with_conditions(content)
        except FileNotFoundError:
            print(f"Błąd: Plik {file_path} nie został znaleziony.")
            return None
        except Exception as e:
            print(f"Wystąpił błąd podczas przetwarzania pliku: {str(e)}")
            return None
        
parser = MultiplicationParser()
text = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)do()?mul(8,5))"
result = parser.print_results_with_conditions(text)

Kolejność instrukcji:
Pozycja -1: do
Pozycja 1: mul
Pozycja 20: dont
Pozycja 28: mul
Pozycja 48: mul
Pozycja 57: do
Pozycja 62: mul
Włączono mnożenia na pozycji -1
Mnożenie na pozycji 1: 2 * 4 = 8 (włączone)
Wyłączono mnożenia na pozycji 20
Włączono mnożenia na pozycji 57
Mnożenie na pozycji 62: 8 * 5 = 40 (włączone)

Znalezione mnożenia (tylko włączone):
2 * 4 = 8 (włączone)
8 * 5 = 40 (włączone)

Suma wszystkich włączonych iloczynów: 48


In [8]:
result = parser.process_file("input.txt")

Kolejność instrukcji:
Pozycja -1: do
Pozycja 12: mul
Pozycja 30: mul
Pozycja 52: mul
Pozycja 76: mul
Pozycja 97: do
Pozycja 144: mul
Pozycja 177: mul
Pozycja 208: mul
Pozycja 232: mul
Pozycja 252: mul
Pozycja 269: do
Pozycja 283: mul
Pozycja 326: do
Pozycja 349: mul
Pozycja 371: mul
Pozycja 423: mul
Pozycja 457: dont
Pozycja 464: mul
Pozycja 499: mul
Pozycja 524: mul
Pozycja 545: mul
Pozycja 557: mul
Pozycja 578: mul
Pozycja 603: mul
Pozycja 620: mul
Pozycja 631: mul
Pozycja 645: mul
Pozycja 688: do
Pozycja 695: mul
Pozycja 720: mul
Pozycja 770: mul
Pozycja 785: mul
Pozycja 809: mul
Pozycja 827: mul
Pozycja 865: mul
Pozycja 878: dont
Pozycja 900: mul
Pozycja 912: do
Pozycja 925: mul
Pozycja 940: mul
Pozycja 975: mul
Pozycja 987: mul
Pozycja 1008: mul
Pozycja 1025: mul
Pozycja 1046: mul
Pozycja 1060: dont
Pozycja 1070: mul
Pozycja 1114: mul
Pozycja 1129: mul
Pozycja 1152: mul
Pozycja 1175: mul
Pozycja 1190: do
Pozycja 1199: mul
Pozycja 1212: mul
Pozycja 1231: dont
Pozycja 1241: mul
Pozy

Your puzzle answer was 83158140.

Both parts of this puzzle are complete! They provide two gold stars: **